# B. Configure MintPy Time Series Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

The configuration for a MintPy Time Series Analysis is defined in a config file called [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/main/src/mintpy/defaults/smallbaselineApp.cfg).

This file contains all optional parameters with their default settings.

When using MintPy, do not edit this file directly. Instead, you will create a custom config file that contains only the parameters that you wish to update. MintPy will load the custom config, using it to overwrite default settings in `smallbaselineApp.cfg`.

**You must reprocess the time series after making configuration updates**

You can update the configuration without needing to reload the source input data. However, you must rerun the `modify_network` step to register the updated configuration, and then rerun the remaining [smallbaselineApp.py](https://github.com/insarlab/MintPy/blob/main/src/mintpy/smallbaselineApp.py) steps to reprocess the time series with your updates.

If you make updates in this notebook, skip reprocessing the time series, and jump directly to re-plotting or outputting to GeoTiff, you will see results from the previous time series, not from your most recent updates. 

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
from ipyfilechooser import FileChooser
import os
from pathlib import Path
from pprint import pprint

import ipywidgets as widgets
import mintpy
import opensarlab_lib as osl
import re
from util import util

---
## 1. Examine Current State of Custom Config File

**Select your custom config file**

- It will be located in your loaded MintPy time series' `MintPy` directory
- It is a text file with your MintPy project name
  - ie., `path/to/MintPy/my_project.txt`

In [ ]:
file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

**Open and display the current contents of the custom config**

In [ ]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

custom_config_dict = {}

custom_config_path = Path(fc.selected)
with open(custom_config_path, 'r') as f:
    custom_lines = f.readlines()

custom_lines = [l for l in custom_lines if not l.startswith('\n') and not l.strip().startswith('#')]

for l in custom_lines:
    param = l.split(' ')[0]
    info = l.split('= ')[-1].strip()
    custom_config_dict[param] = info

print('Current custom config:\n')
for l in custom_lines:
    print(l)

---
## 2. Select how you would like to update your time series configuration

In [ ]:
config_option_select = osl.select_parameter(
    [
        "Select a few commonly adjusted options",
        "View the full (very long) MintPy config to set any available option"
    ]
)
display(config_option_select)

---
## 3. Update the Current Configuration 

In [ ]:
full_config = 'very long' in config_option_select.value

if not full_config:
    cpu_count = os.cpu_count()
    multithread_option = osl.select_parameter(["Do not use multithreaded processing",
                                         f"Use my {cpu_count} available cores for multithreaded processing"],
                                             description="Select a multithreaded processing option:")
    
    ref_point_option = osl.select_parameter(["Allow MintPy to determine a reference point", 
                                             "Define a reference point"],
                                           description="Select a refernce point option:")

    ref_date_option = osl.select_parameter(["Reference time-series to earliest date in stack",
                                            "Allow MintPy to determine reference date"],
                                          description="Select a reference date option:")

    tropo_correct_option = osl.select_parameter(["Do not perform tropospheric correction",
                                            "Perform tropospheric correction"],
                                          description="Select a tropospheric correction option:")
    deramp_option = osl.select_parameter(["Do not perform deramping",
                                          "Deramp method: linear",
                                          "Deramp method: quadratic"
                                         ],
                                          description="Select a phase deramping option:")

    display(multithread_option)
    display(ref_point_option)
    display(ref_date_option)
    display(tropo_correct_option)
    display(deramp_option)

In [ ]:
if not full_config:
    updated_config = []
    multithread = 'Use' in multithread_option.value
    mintpy_ref_point = 'Allow' in ref_point_option.value
    mintpy_ref_date = 'Allow' in ref_date_option.value
    tropo_correct = 'Do not' not in tropo_correct_option.value
    deramp = 'Do not' not in deramp_option.value

    for l in custom_lines:
        if all(x not in l for x in ['compute', 'reference', 'troposphericDelay', 'deramp']):
            updated_config.append(l)

    if multithread:
        updated_config.append('mintpy.compute.cluster = local')
        updated_config.append(f'mintpy.compute.numWorker = {cpu_count}')
        
    if not mintpy_ref_date:
        updated_config.append(f'mintpy.reference.date = no')
        
    if tropo_correct:
        updated_config.append('mintpy.troposphericDelay.method = pyaps')
    else:
        updated_config.append('mintpy.troposphericDelay.method = no')

    if not mintpy_ref_point:
        is_float = False
        while not is_float:
            try:
                lat = float(input("Enter reference latitude"))
                lon = float(input("Enter reference longitude"))
                is_float = True
            except ValueError:
                print("Latitude and Longitude must be convertable to float")
                continue
            updated_config.append(f'mintpy.reference.lalo = {lat},{lon}')

    if deramp and 'linear' in deramp_option.value:
        deramp_method = 'linear'
    elif deramp and 'quadratic' in deramp_option.value:
        deramp_method = 'quadratic'
    else:
        deramp_method = 'no'
    updated_config.append(f'mintpy.deramp = {deramp_method}')

    updated_config = '\n'.join(updated_config)
    

In [ ]:
if not full_config:
    print('Current updated custom config:\n')
    pprint(updated_config)
    print('Proceed to the end of the notebook to save the updated config.')

**Read in the default config**

In [ ]:
if full_config:
    default_config_path = Path(mintpy.__file__).parent / "defaults/smallbaselineApp.cfg"
    with open(default_config_path, 'r') as f:
        lines = f.readlines()
        
    lines = [l for l in lines if not l.startswith('\n')]

**Complete an interactive config form**

***Only complete the fields you wish to change***

- The form contains settings from your existing custom config and parameter hints for any unused attributes
- Uncompleted fields will be set to 'auto'

**Some Commonly Updated Options:**

- Set a reference point
  -  `mintpy.reference.lalo`
  -  `mintpy.reference.yx`
- Use the time series start date as the reference date
  - `mintpy.reference.date = no`
- Use multihreaded processing for inversion and DEM error correction
  - `mintpy.compute.cluster = local`
  - `mintpy.compute.numWorker`
- Subset
  - `mintpy.subset.lalo`
  - `mintpy.subset.yx`
- Tropospheric correction (optional)
  - `mintpy.troposphericDelay.method = pyaps`
  - `mintpy.troposphericDelay.method = no`
- Phase deramping (optional)
  - `mintpy.deramp = linear`
  - `mintpy.deramp = quadratic`
  - `mintpy.deramp = no` == `mintpy.deramp = auto`
    

In [ ]:
if full_config:
    param_dict = {}
    
    layout = widgets.Layout(width='initial', height='40px') #set width and height
    
    for l in lines:
        if l.startswith('#'):
            print(l)
        else:
            param = l.split(' ')[0]
            if param in custom_config_dict.keys():
                info = custom_config_dict[param]
            else:
                info = l.split('= auto ')[-1].strip()[1:]  
            param_dict[param] =  widgets.Text(
                placeholder=info,
                description=f'{param}:',
                disabled=False,
                align_items='stretch', 
                layout = layout,
                style= {'description_width': 'initial'},
            )
            display(param_dict[param])

**Create updated list of custom configuration settings**

- Check your settings before overwriting your config file
- Any settings not included in your custom config will be treated as `auto` by MintPy

In [ ]:
if full_config:
    updated_config = ""
    
    for k, v in param_dict.items():
        if len(v.value) > 0 and v.value != v.placeholder:
            updated_config += f'{k} = {v.value}\n'
        elif k in custom_config_dict.keys() and v.placeholder == custom_config_dict[k]:
            updated_config += f'{k} = {v.placeholder}\n'
    print('Updated custom config settings:\n')
    pprint(updated_config)

**Overwrite your custom config with updated settings**

- If you are happy with the output from the previous cell, overwrite your config file

In [ ]:
custom_config_path.write_text(updated_config)

**It is now time to run the `c_MintPy_Time_Series.ipynb` (Perform MintPy Time Series Analysis) Notebook**

If you skip rerunning the time series and create plots or Geotiff output now, the changes you just made will not be applied. 